# Effective Jsat Calculations - PoC only

$J_{sat,eff} := \frac{E_{sat}}{\pi w^2} \text{ where } E_{sat}:= E$ value such that loss due to saturable absorption becomes half of its initial value

Let us use binary search to find it for some configurations and see how this new parameter behaves

In [1]:
from beam import *

In [3]:
Einput = np.pi*(25**2)*0.00000015*10  # Einput will be 10-fold of [Jsat*Area]
ee = beam_initialize(res=33, threshold=10**-10, w=25, Ep=Einput)  # Resolution: about 30nm for unit length
Ein = integrate_for_energy(ee)
graphene_config = [[5,0],[5,5],[5,10],[5,15]]

results = []
for i in graphene_config:
    mm = mask_initialize(beam=ee, shape='lines',width=i[0],thickness=i[1], a0=0)  #a0 zeroed out
    # Initial ΔΕ/Ε calculation:
    ss = mask_apply(beam=ee,mask=mm)
    initEfrac_over2 = ((Ein-integrate_for_energy(ss))/Ein)/2
    # Searching:
    cursor = 5*Ein  # Change cursor&bound as you please
    bound = 5*Ein  # Will search around [0, 10*Ein]
    while True:  # Binary search, might get caught up in an infinite loop if boundaries are not sane
        # Binary search between E = [Ein,0], if ΔΕ/Ε approx (initEfrac/2), append result to results[]
        # Percent difference threshold for bingo: <1%
        ee_tmp = beam_initialize(res=33, threshold=10**-10, w=25, Ep=cursor)
        mm = mask_initialize(beam=ee_tmp, shape='lines',width=i[0],thickness=i[1],a0=0)
        ss = mask_apply(beam=ee_tmp,mask=mm)
        Eval = integrate_for_energy(ee_tmp)
        tmp_dE_E = (Eval - integrate_for_energy(ss))/Eval
        if (abs(tmp_dE_E-initEfrac_over2)/initEfrac_over2)<0.01:  # Toggle 0.01 to change threshold
            results.append(Eval/(np.pi*(25**2)))
            print(f"E_sat found! : {Eval}")
            print(f"J_sateff ---> {Eval/(np.pi*(25**2))} uJ/um2 for E_p={Ein} uJ")
            print(f"Graphene Layer configuration: {i}, a_s={mm.aS}")
            break
        elif tmp_dE_E > initEfrac_over2:
            bound /= 2
            cursor += bound
        else:
            bound /= 2
            cursor -= bound
        print(f"Cursor at: {cursor} uJ")


Cursor at: 0.007377111469395459 uJ
Cursor at: 0.011065667204093188 uJ
Cursor at: 0.009221389336744323 uJ
Cursor at: 0.008299250403069891 uJ
Cursor at: 0.007838180936232675 uJ
Cursor at: 0.007607646202814067 uJ
E_sat found! : 0.007622269913554192
J_sateff ---> 3.881990189832907e-06 uJ/um2 for E_p=0.0029508445877581835 uJ
Graphene Layer configuration: [5, 0], a_s=0.00575
Cursor at: 0.007377111469395459 uJ
Cursor at: 0.011065667204093188 uJ
Cursor at: 0.009221389336744323 uJ
Cursor at: 0.008299250403069891 uJ
Cursor at: 0.007838180936232675 uJ
Cursor at: 0.007607646202814067 uJ
E_sat found! : 0.007622269913554192
J_sateff ---> 3.881990189832907e-06 uJ/um2 for E_p=0.0029508445877581835 uJ
Graphene Layer configuration: [5, 5], a_s=0.00575
Cursor at: 0.007377111469395459 uJ
Cursor at: 0.011065667204093188 uJ
Cursor at: 0.009221389336744323 uJ
Cursor at: 0.008299250403069891 uJ
Cursor at: 0.007838180936232675 uJ
Cursor at: 0.007607646202814067 uJ
E_sat found! : 0.007622269913554192
J_sateff -

Would $J_{sat,eff}$ be useful?